In [69]:
%%file encoding.lp
%%clingo 0 instances/env2.lp -
%%file encoding.lp
%%clingo 0 instances/env4.lp -



%#const n=5.

%_________________Encode Tracks____________________

%straight tracks
track(1025, (1, 3)). track(32800, (0, 2)). track(4608, (3, 2)). track(16386, (2, 1)). track(72, (0, 1)). track(2064, (0, 3)).

%simple switch
track(3089, (3, 1)). track(3089, (3, 0)). track(37408, (3, 2)). track(37408, (0, 2)). track(17411, (2, 3)). track(17411, (1, 3)).
track(32872, (0, 1)). track(32872, (0, 2)). track(49186, (0, 2)). track(49186, (2, 1)). track(1097, (1, 3)). track(1097, (0, 1)).
track(34864, (3, 0)). track(34864, (0, 2)). track(5633, (3, 1)). track(5633, (3, 2)).

%diamond crossing
track(33825, (0, 2)). track(33825, (1, 3)).

%diamond plus crossing
track(38433, (0, 2)). track(38433, (1, 3)). track(38433, (3, 2)). track(50211, (0, 2)). track(50211, (1, 3)). track(50211, (2, 1)).
track(33897, (0, 2)). track(33897, (1, 3)). track(33897, (0, 1)). track(35889, (0, 2)). track(35889, (1, 3)). track(35889, (0, 3)).
track(38505, (0, 2)). track(38505, (1, 3)). track(38505, (3, 2)). track(38505, (0, 1)).
track(52275, (0, 2)). track(52275, (1, 3)). track(52275, (2, 1)). track(52275, (0, 3)).

%double curve crossing
track(20994, (1, 2)). track(20994, (2, 3)). track(16458, (0, 1)). track(16458, (1, 2)). 
track(2136, (0, 1)). track(2136, (3, 0)). track(6672, (2, 3)). track(6672, (3, 0)).

%track end
%track(8192, (2, end)). track(4, (1, end)). track(128, (0, end)). track(256, (3, end)).

track(N, (X, Y)) :- track(N, (Y, X)).

moveDir(0, (0, 1)). %go north 
moveDir(1, (1, 0)). %go east
moveDir(2, (0, -1)). %go south
moveDir(3, (-1, 0)). %go west

revDir(0, 2). revDir(2, 0). revDir(1, 3). revDir(3, 1).

dir2int(n, 0).
dir2int(e, 1).
dir2int(s, 2).
dir2int(w, 3).

%move left: track turns left, other option exists
move(Type, DirL, move_left):- track(Type, (DirIn, DirL)), track(Type, (DirIn, DirR)), DirL = DirIn + 1, DirL != DirR.
move(Type, 0, move_left):- track(Type, (3, 0)), track(Type, (3, DirR)), DirR > 0.

%move right: track turns right, other option exists
move(Type, DirR, move_right):- track(Type, (DirIn, DirL)), track(Type, (DirIn, DirR)), DirR = DirIn - 1, DirL != DirR.
move(Type, 3, move_right):- track(Type, (0, 3)), track(Type, (0, DirL)), DirL < 3.

    
move(Type, DirOut, move_forward) :- track(Type, (DirIn, DirOut)), |DirOut - DirIn| = 2.
move(Type, DirOut, move_forward) :- track(Type, (DirIn, DirOut)),
    not move(Type, DirOut, move_left),
    not move(Type, DirOut, move_right).    

%_________________Generate__________________________

agent(A) :- start(agent(A), _, _, _).
action_type(move_forward). action_type(wait). action_type(move_left). action_type(move_right).

#program base 
action(agent(A), wait, 0) :- agent(A).

#program step(t)    
{action(agent(A), Action, t)} :- start(agent(A),(Y,X), _, dir(D)), action_type(Action), time(t).


%one action for each agent and each timestamp
:- t, agent(A), not 1 {action(agent(A), _, t)} 1.

% cant do anything other than wait before earliert departure time
:- t, start(agent(A), _, EarliestDeparture, _), current_state(agent(A), (Y,X), t, DirOut), EarliestDeparture > t, not action(agent(A), wait, t).
     
current_state(agent(A), (Y,X), 0, Dir) :- start(agent(A),(Y,X),_,dir(D)), dir2int(D, Dir).  
    
current_state(agent(A), (Y + DY, X+DX), t + 1, DirOut2) :- current_state(agent(A), (Y,X), t, DirOut),
    action(agent(A), Action, t + 1), cell((Y,X), Type), cell((Y + DY, X+DX), Type2),
    track(Type, (_, DirOut)), track(Type2, (DirIn,DirOut2)), move(Type, DirOut, Action),
    revDir(DirIn, DirOut), moveDir(DirOut, (DY,DX)).

current_state(agent(A), (Y,X), t + 1, Dir) :- current_state(agent(A), (Y,X), T, Dir), action(agent(A), wait, t + 1).
    
:- action(agent(A), _, t), not current_state(agent(A), _, t, _).

%vertex conflict
:-current_state(agent(A1), (Y,X), t, _), current_state(agent(A2), (Y,X), t, _), A1 < A2.

%also include edge conflict later 
    

#program check(t)
:- end(agent(A), (Y,X), LatestArrival), current_state(agent(A), (Y,X), query(t), _), t > LatestArrival.
    
    
#show action/3.
%#show current_state/4.
%#show move/3.
%#show track/2.

Overwriting encoding.lp


In [81]:
%%file inc.py
import sys
from clingo.symbol import Function, Number
from clingo.application import Application, clingo_main
class IncConfig:
    def init(self):
        self.imin, self.imax, self.istop = 1, None, "SAT"
# [...]

class IncExampleApp(Application):
    program_name = "inc-example"
    version = "1.0"
    def init(self):
        self._conf = IncConfig()
    def register_options(self, options):
        group = "Inc - Example Options"
        options.add(
            group, "imin",
            f"Minimum number of steps [{self._conf.imin}] ",
            parse_int(self._conf, "imin", min_val=0) ,
            argument="<n>")
        options.add(
            group, " imax ",
            f"Maximum number of steps [{ self._conf.imax }] ",
            parse_int(self._conf, "imax", min_val=0 , optional=True),
            argument="<n>")
        options.add(
            group, "istop",
            f"Stop criterion [{ self._conf.istop}] ",
            parse_stop(self._conf, "istop"))

    def main(self, ctl, files ):
        if not files: files = ["-"]
        for f in files: ctl.load(f)
        ctl.add("check", ["t"], "#external query ( t ).")

        conf = self._conf
        imin, imax, istop = conf.imin, conf.imax, conf.istop

        step, ret=0, None
        while((imax is None or step < imax) and
                (step == 0 or step < imin or (
                    (istop == "SAT" and not ret.satisfiable) or
                    (istop == "UNSAT" and not ret.unsatisfiable) or
                    (istop == "UNKNOWN" and not ret.unknown)))):
            parts = []
            parts.append(("check", [Number(step)]))
            if step > 0:
                query = Function ("query", [Number(step - 1)])
                ctl.release_external(query)
                parts.append(("step", [Number(step)]))
            else :
                parts.append(("base", []))
            ctl.ground(parts)
            query = Function("query", [Number(step)])
            ctl.assign_external(query, True)
            ret, step = ctl.solve(), step+1
clingo_main(IncExampleApp(), sys.argv[1:])

Overwriting inc.py


In [82]:
!python inc.py encoding.lp instances/env2.lp

Traceback (most recent call last):
  File "/Users/ekaterinavolosinova/opt/anaconda3/envs/potassco2/lib/python3.11/site-packages/clingo/application.py", line 376, in _pyclingo_application_register_options
    app.register_options(ApplicationOptions(options, mem))
  File "/Users/ekaterinavolosinova/ASP/train-scheduling/inc.py", line 18, in register_options
    f"Minimum number of steps [{self._conf.imin}] ",
                                ^^^^^^^^^^
AttributeError: 'IncExampleApp' object has no attribute '_conf'


In [52]:
s1 ="move(32872,1,move_left) move(38505,1,move_left) move(52275,2,move_left) move(20994,3,move_left) move(2136,1,move_left) move(16458,2,move_left) move(38505,3,move_left) move(37408,3,move_left) move(3089,0,move_left) move(6672,0,move_left) move(52275,0,move_left) move(49186,1,move_right) move(5633,2,move_right) move(52275,1,move_right) move(20994,1,move_right) move(6672,2,move_right) move(16458,0,move_right) move(38505,0,move_right) move(38505,2,move_right) move(1097,0,move_right) move(17411,2,move_right) move(52275,3,move_right) move(2136,3,move_right) move(34864,3,move_right) move(1025,3,move_forward) move(32800,2,move_forward) move(3089,1,move_forward) move(37408,2,move_forward) move(17411,3,move_forward) move(32872,2,move_forward) move(49186,2,move_forward) move(1097,3,move_forward) move(34864,2,move_forward) move(5633,1,move_forward) move(33825,2,move_forward) move(33825,3,move_forward) move(38505,2,move_forward) move(38505,3,move_forward) move(52275,2,move_forward) move(52275,3,move_forward) move(52275,1,move_forward) move(52275,0,move_forward) move(38505,1,move_forward) move(38505,0,move_forward) move(33825,1,move_forward) move(33825,0,move_forward) move(5633,3,move_forward) move(34864,0,move_forward) move(1097,1,move_forward) move(49186,0,move_forward) move(32872,0,move_forward) move(17411,1,move_forward) move(37408,0,move_forward) move(3089,3,move_forward) move(32800,0,move_forward) move(1025,1,move_forward) move(4608,2,move_forward) move(16386,1,move_forward) move(72,1,move_forward) move(2064,3,move_forward) move(16458,1,move_forward) move(6672,3,move_forward) move(2136,0,move_forward) move(20994,2,move_forward) move(2064,0,move_forward) move(72,0,move_forward) move(16386,2,move_forward) move(4608,3,move_forward)"

In [58]:
b = s1.split()
b.sort()
a = s.split()
a.sort()


In [59]:
len(a), len(b)

(92, 68)

In [60]:
for i in range(len(a)):
    if i < len(b):
        print(a[i], b[i])
    else:
        print(a[i])

move(1025,1,move_forward) move(1025,1,move_forward)
move(1025,3,move_forward) move(1025,3,move_forward)
move(1097,0,move_right) move(1097,0,move_right)
move(1097,1,move_forward) move(1097,1,move_forward)
move(1097,3,move_forward) move(1097,3,move_forward)
move(16386,1,move_forward) move(16386,1,move_forward)
move(16386,2,move_forward) move(16386,2,move_forward)
move(16458,0,move_right) move(16458,0,move_right)
move(16458,1,move_forward) move(16458,1,move_forward)
move(16458,2,move_left) move(16458,2,move_left)
move(17411,1,move_forward) move(17411,1,move_forward)
move(17411,2,move_right) move(17411,2,move_right)
move(17411,3,move_forward) move(17411,3,move_forward)
move(2064,0,move_forward) move(2064,0,move_forward)
move(2064,3,move_forward) move(2064,3,move_forward)
move(20994,1,move_right) move(20994,1,move_right)
move(20994,2,move_forward) move(20994,2,move_forward)
move(20994,3,move_left) move(20994,3,move_left)
move(2136,0,move_forward) move(2136,0,move_forward)
move(2136,1,move_l

In [56]:
s = "move(32872,1,move_left) move(50211,2,move_left) move(33897,1,move_left) move(38505,1,move_left) move(52275,2,move_left) move(2136,1,move_left) move(16458,2,move_left) move(20994,3,move_left) move(38505,3,move_left) move(38433,3,move_left) move(37408,3,move_left) move(3089,0,move_left) move(6672,0,move_left) move(52275,0,move_left) move(35889,0,move_left) move(49186,1,move_right) move(5633,2,move_right) move(38433,2,move_right) move(50211,1,move_right) move(38505,2,move_right) move(52275,1,move_right) move(6672,2,move_right) move(16458,0,move_right) move(20994,1,move_right) move(38505,0,move_right) move(33897,0,move_right) move(1097,0,move_right) move(17411,2,move_right) move(35889,3,move_right) move(52275,3,move_right) move(2136,3,move_right) move(34864,3,move_right) move(1025,3,move_forward) move(32800,2,move_forward) move(3089,1,move_forward) move(37408,2,move_forward) move(17411,3,move_forward) move(32872,2,move_forward) move(49186,2,move_forward) move(1097,3,move_forward) move(34864,2,move_forward) move(5633,1,move_forward) move(33825,2,move_forward) move(33825,3,move_forward) move(38433,2,move_forward) move(38433,3,move_forward) move(50211,2,move_forward) move(50211,3,move_forward) move(33897,2,move_forward) move(33897,3,move_forward) move(35889,2,move_forward) move(35889,3,move_forward) move(38505,2,move_forward) move(38505,3,move_forward) move(52275,2,move_forward) move(52275,3,move_forward) move(52275,1,move_forward) move(52275,0,move_forward) move(38505,1,move_forward) move(38505,0,move_forward) move(35889,1,move_forward) move(35889,0,move_forward) move(33897,1,move_forward) move(33897,0,move_forward) move(50211,1,move_forward) move(50211,0,move_forward) move(38433,1,move_forward) move(38433,0,move_forward) move(33825,1,move_forward) move(33825,0,move_forward) move(5633,3,move_forward) move(34864,0,move_forward) move(1097,1,move_forward) move(49186,0,move_forward) move(32872,0,move_forward) move(17411,1,move_forward) move(37408,0,move_forward) move(3089,3,move_forward) move(32800,0,move_forward) move(1025,1,move_forward) move(4608,2,move_forward) move(16386,1,move_forward) move(72,1,move_forward) move(2064,3,move_forward) move(20994,2,move_forward) move(16458,1,move_forward) move(2136,0,move_forward) move(6672,3,move_forward) move(2064,0,move_forward) move(72,0,move_forward) move(16386,2,move_forward) move(4608,3,move_forward)"